<a href="https://colab.research.google.com/github/vlad-pirvu/LLM_testing/blob/main/D%26D_Spells.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### I.Dependencies & Imports

In [1]:
import nbformat
from google.colab import _message
import os

def strip_widgets_metadata():
    # Load current notebook's contents
    notebook_content = _message.blocking_request('get_ipynb')['ipynb']
    nb = nbformat.from_dict(notebook_content)

    # Recursively remove 'widgets' key from dictionary
    def remove_widgets_recursive(obj):
        if isinstance(obj, dict):
            keys_to_delete = [key for key in obj if key == 'widgets']
            for key in keys_to_delete:
                del obj[key]
            for value in obj.values():
                remove_widgets_recursive(value)
        elif isinstance(obj, list):
            for item in obj:
                remove_widgets_recursive(item)

    # Apply recursive removal to notebook structure
    remove_widgets_recursive(nb)


    # Overwrite the current notebook file
    try:
        notebook_path = ipynbname.path()
        with open(notebook_path, 'w', encoding='utf-8') as f:
            nbformat.write(nb, f)
        print(f'✅ Cleaned widgets metadata from {notebook_path}')
    except FileNotFoundError:
        print("❌ Could not determine notebook path. Please ensure the notebook is saved.")
    except Exception as e:
        print(f"❌ An error occurred while saving the notebook: {e}")


strip_widgets_metadata()

❌ An error occurred while saving the notebook: name 'ipynbname' is not defined


In [2]:
# Install dependencies
!pip install -q requests beautifulsoup4 sentence-transformers faiss-cpu gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.9 MB/s eta 0:00:00


In [3]:
!pip install tqdm

In [4]:
!pip install transformers sentence-transformers

In [5]:
# Import libraries
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import gradio as gr
from tqdm import tqdm
import pandas as pd

### II. Scrape D&D 5e Spells from D&D Wiki

In [6]:
# Step 2: Scrape D&D 5e Spells
BASE_URL = "https://dnd5e.wikidot.com"

In [7]:
def get_spell_links():
    res = requests.get(f"{BASE_URL}/spells")
    soup = BeautifulSoup(res.text, 'html.parser')
    links = [a['href'] for a in soup.select('div#page-content a') if a['href'].startswith('/spell:')]
    return list(set(links))

In [8]:
def scrape_spell(link):
    base_url = "https://dnd5e.wikidot.com"
    if not link.startswith("http"):
        url = base_url + link
    else:
        url = link

    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    # Try finding the spell name in multiple ways
    title_tag = soup.find('div', {'id': 'page-title'})
    if not title_tag:
        title_tag = soup.find('div', class_='page-title page-header')
    if title_tag:
        # If using class selector, the name is in a <span> inside the div
        if title_tag.name == 'div' and 'page-header' in title_tag.get('class', []):
            name = title_tag.find('span').text.strip()
        else:
            name = title_tag.text.strip()
    else:
        print(f"⚠️ Skipping {url}: no title found.")
        return None

    # Skip Unearthed Arcana spells
    if name.endswith(" UA") or name.endswith(" (UA)"):
        return None

    # Get the spell content
    content_tag = soup.find('div', {'id': 'page-content'})
    if not content_tag:
        print(f"⚠️ Skipping {url}: no page-content found.")
        return None
    content = content_tag.text.strip()

    return {'Url': url, 'Name': name, 'content': content}


In [9]:
spell_links = get_spell_links()
print(f"Found {len(spell_links)} spells. Scraping...")

# Collect spells
spells_data = []
for link in tqdm(spell_links, desc="Scraping spells"):
#    full_url = BASE_URL + link
    spell = scrape_spell(link)
    if spell:
        spells_data.append(spell)

Found 580 spells. Scraping...


Scraping spells: 100%|██████████| 580/580 [02:34<00:00,  3.75it/s]


### III. Format Spells DataFrame

In [10]:
# Convert to DataFrame
spells_df = pd.DataFrame(spells_data)
print(f"✅ Collected {len(spells_df)} spells.")
spells_df.head()

✅ Collected 530 spells.


,Url,Name,content
0,https://dnd5e.wikidot.com/spell:conjure-minor-...,Conjure Minor Elementals,Source: Player's Handbook\n4th-level conjurati...
1,https://dnd5e.wikidot.com/spell:conjure-barrage,Conjure Barrage,Source: Player's Handbook\n3rd-level conjurati...
2,https://dnd5e.wikidot.com/spell:word-of-radiance,Word of Radiance,Source: Xanathar's Guide to Everything\nEvocat...
3,https://dnd5e.wikidot.com/spell:otilukes-freez...,Otiluke's Freezing Sphere,Source: Player's Handbook\n6th-level evocation...
4,https://dnd5e.wikidot.com/spell:command,Command,Source: Player's Handbook\n1st-level enchantme...


In [11]:
spells_df.shape

(530, 3)

In [12]:
def split_spell_content(row):
    #print("------")
    #print(row.name)
    #print(row['name'])

    lines = row['content'].split('\n')
    lines = [line.strip() for line in lines if line.strip()]  # Clean up empty lines and whitespace

    # Safe defaults
    source = lines[0].replace("Source", "").replace("source","").lstrip(',.:;-') if len(lines) > 0 else ''
    spell_level_type = lines[1].replace("Level", "").replace("level", "").replace("-","").replace("st","").replace("nd","").replace("rd","").replace("th","").lstrip(',.:;-') if len(lines) > 1 else ''
    casting_time = lines[2].replace("Casting Time", "").replace("Casting time","").replace("casting time","").lstrip(',.:;-') if len(lines) > 2 else ''
    spell_range = lines[3].replace("Range", "").replace("range","").lstrip(',.:;-') if len(lines) > 3 else ''
    components = lines[4].replace("Components", "").replace("components","").lstrip(',.:;-') if len(lines) > 4 else ''
    duration = lines[5].replace("Duration", "").replace("duration","").lstrip(',.:;-') if len(lines) > 5 else ''

    # Split spell_level_type into level and type
    if 'cantrip' in spell_level_type.lower():
        level, school = '0', spell_level_type.replace('cantrip', '').strip()
    elif ' ' in spell_level_type:
        level, school = spell_level_type.split(' ', 1)
    else:
        level, school = '', spell_level_type


    #Extract description
    mashed = row['content'].split('Duration',1)[1]
    mashed = mashed.split('\n',1)[1].strip().lower()

    if 'at higher levels' in mashed:
        description = mashed.split('at higher levels')[0].strip()
        upcastingMashed = mashed.split('at higher levels')[1].lstrip(',.:;-').strip()
        upcasting = upcastingMashed.split('spell lists')[0].strip()
        spellLists = upcastingMashed.split('spell lists')[1].lstrip(',.:;-').strip()
    else:
        description = mashed.split('spell lists')[0].strip()
        mashed = mashed.replace(":",".")
        spellLists = mashed.split('spell lists')[1].lstrip(',.:;-').strip()
        upcasting = ""

    return pd.Series({
        'Source': source,
        'Level': level,
        'School': school,
        'Casting Time': casting_time,
        'Range': spell_range,
        'Components': components,
        'Duration': duration,
        'Description': description,
        'Upcasting': upcasting,
        'Spell Lists': spellLists
    })


In [13]:
# Apply the split function to the spells DataFrame
spell_df_split = spells_df.join(spells_df.apply(split_spell_content, axis=1))

In [14]:
spell_df_split.head()

,Url,Name,content,Source,Level,School,Casting Time,Range,Components,Duration,Description,Upcasting,Spell Lists
0,https://dnd5e.wikidot.com/spell:conjure-minor-...,Conjure Minor Elementals,Source: Player's Handbook\n4th-level conjurati...,Player's Handbook,4,conjuration,1 minute,90 feet,"V, S","Concentration, up to 1 hour",you summon elementals that appear in unoccupie...,when you cast this spell using certain higher-...,"druid, wizard"
1,https://dnd5e.wikidot.com/spell:conjure-barrage,Conjure Barrage,Source: Player's Handbook\n3rd-level conjurati...,Player's Handbook,3,conjuration,1 action,Self (60-foot cone),"V, S, M (one piece of ammunition or a thrown ...",Instantaneous,you throw a nonmagical weapon or fire a piece ...,,ranger
2,https://dnd5e.wikidot.com/spell:word-of-radiance,Word of Radiance,Source: Xanathar's Guide to Everything\nEvocat...,Xanathar's Guide to Everything,0,Evocation,1 action,5 feet,"V, M (a holy symbol)",Instantaneous,"you utter a divine word, and burning radiance ...",the spell’s damage increases by 1d6 when you r...,cleric
3,https://dnd5e.wikidot.com/spell:otilukes-freez...,Otiluke's Freezing Sphere,Source: Player's Handbook\n6th-level evocation...,Player's Handbook,6,evocation,1 action,300 feet,"V, S, M (a small crystal sphere)",Instantaneous,a frigid globe of cold energy streaks from you...,when you cast this spell using a spell slot of...,"sorcerer (optional), wizard"
4,https://dnd5e.wikidot.com/spell:command,Command,Source: Player's Handbook\n1st-level enchantme...,Player's Handbook,1,enchantment,1 action,60 feet,V,1 round,you speak a one-word command to a creature you...,when you cast this spell using a spell slot of...,"bard (optional), cleric, paladin"


In [15]:
spell_df_split.describe()

,Url,Name,content,Source,Level,School,Casting Time,Range,Components,Duration,Description,Upcasting,Spell Lists
count,530,530,530,530,530,530,530,530,530,530,530,530,530
unique,530,530,530,17,10,34,20,37,276,32,530,156,161
top,https://dnd5e.wikidot.com/spell:unseen-servant,Unseen Servant,Source: Player's Handbook\n1st-level conjurati...,Player's Handbook,2,evocation,1 action,60 feet,"V, S",Instantaneous,"this spell creates an invisible, mindless, sha...",,wizard
freq,1,1,1,361,88,90,395,96,161,148,1,329,41


In [16]:
non_numeric_levels = spell_df_split[~spell_df_split['Level'].str.isdigit()]['Level'].unique()
print("Non-numeric values in the 'Level' column:", non_numeric_levels)

Non-numeric values in the 'Level' column: []


In [17]:
spell_df_split = spell_df_split.drop('content', axis=1)

In [18]:
spell_df_split = spell_df_split.rename(columns={'name': 'Name', 'Url':"URL"})
print(spell_df_split.columns)

Index(['URL', 'Name', 'Source', 'Level', 'School', 'Casting Time', 'Range',
       'Components', 'Duration', 'Description', 'Upcasting', 'Spell Lists'],
      dtype='object')


In [19]:
desired_column_order = ['Name', 'Level', 'School', 'Casting Time', 'Range', 'Components', 'Duration', 'Description', 'Upcasting', 'Spell Lists', 'Source', 'URL']

In [20]:
finalSpells = spell_df_split[desired_column_order]

In [21]:
finalSpells.head()

,Name,Level,School,Casting Time,Range,Components,Duration,Description,Upcasting,Spell Lists,Source,URL
0,Conjure Minor Elementals,4,conjuration,1 minute,90 feet,"V, S","Concentration, up to 1 hour",you summon elementals that appear in unoccupie...,when you cast this spell using certain higher-...,"druid, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:conjure-minor-...
1,Conjure Barrage,3,conjuration,1 action,Self (60-foot cone),"V, S, M (one piece of ammunition or a thrown ...",Instantaneous,you throw a nonmagical weapon or fire a piece ...,,ranger,Player's Handbook,https://dnd5e.wikidot.com/spell:conjure-barrage
2,Word of Radiance,0,Evocation,1 action,5 feet,"V, M (a holy symbol)",Instantaneous,"you utter a divine word, and burning radiance ...",the spell’s damage increases by 1d6 when you r...,cleric,Xanathar's Guide to Everything,https://dnd5e.wikidot.com/spell:word-of-radiance
3,Otiluke's Freezing Sphere,6,evocation,1 action,300 feet,"V, S, M (a small crystal sphere)",Instantaneous,a frigid globe of cold energy streaks from you...,when you cast this spell using a spell slot of...,"sorcerer (optional), wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:otilukes-freez...
4,Command,1,enchantment,1 action,60 feet,V,1 round,you speak a one-word command to a creature you...,when you cast this spell using a spell slot of...,"bard (optional), cleric, paladin",Player's Handbook,https://dnd5e.wikidot.com/spell:command


In [22]:
finalSpells = finalSpells.sort_values(by='Level')

In [23]:
finalSpells.head(50)

,Name,Level,School,Casting Time,Range,Components,Duration,Description,Upcasting,Spell Lists,Source,URL
181,Light,0,Evocation,1 action,Touch,"V, M (a firefly or phosphorescent moss)",1 hour,you touch one object that is no larger than 10...,,"artificer, bard, cleric, sorcerer, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:light
240,Toll the Dead,0,Necromancy,1 action,60 feet,"V, S",Instantaneous,you point at one creature you can see within r...,the spell’s damage increases by one die when y...,"cleric, warlock, wizard",Xanathar's Guide to Everything,https://dnd5e.wikidot.com/spell:toll-the-dead
161,Thunderclap,0,Evocation,1 action,Self (5-foot radius),S,Instantaneous,"you create a burst of thunderous sound, which ...",the spell’s damage increases by 1d6 when you r...,"bard, druid, sorcerer, warlock, wizard, artificer",Xanathar's Guide to Everything,https://dnd5e.wikidot.com/spell:thunderclap
31,Message,0,Transmutation,1 action,120 feet,"V, S, M (a short piece of copper wire)",1 round,you point your finger toward a creature within...,,"artificer, bard, sorcerer, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:message
235,Create Bonfire,0,Conjuration,1 action,60 feet,"V, S","Concentration, up to 1 minute",you create a bonfire on ground that you can se...,the spell’s damage increases by 1d8 when you r...,"artificer, druid, sorcerer, warlock, wizard",Xanathar's Guide to Everything,https://dnd5e.wikidot.com/spell:create-bonfire
478,Dancing Lights,0,Evocation,1 action,120 feet,"V, S, M (a bit of phosphorus or wychwood, or ...","Concentration, up to 1 minute",you create up to four torch-sized lights withi...,,"artificer, bard, sorcerer, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:dancing-lights
123,Magic Stone,0,Transmutation,1 bonus action,Touch,"V, S",1 minute,you touch one to three pebbles and imbue them ...,,"druid, warlock, artificer",Xanathar's Guide to Everything/Elemental Evil...,https://dnd5e.wikidot.com/spell:magic-stone
37,Shillelagh,0,Transmutation,1 bonus action,Touch,"V, S, M (mistletoe, a shamrock leaf, and a cl...",1 minute,the wood of a club or quarterstaff you are hol...,,druid,Player's Handbook,https://dnd5e.wikidot.com/spell:shillelagh
218,Prestidigitation,0,Transmutation,1 action,10 feet,"V, S",Up to 1 hour,this spell is a minor magical trick that novic...,,"artificer, bard, sorcerer, warlock, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:prestidigitation
39,Fire Bolt,0,Evocation,1 action,120 feet,"V, S",Instantaneous,you hurl a mote of fire at a creature or objec...,this spell’s damage increases by 1d10 when you...,"artificer, sorcerer, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:fire-bolt


### IV. The real deal

In [24]:
#Step IV.1 - Prepare the data to feed to the model

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Create a 'searchable_text' column
finalSpells['searchable_text'] = finalSpells.apply(lambda row: f"{row['Name']} {row['Level']} {row['School']} {row['Casting Time']} {row['Range']} {row['Components']} {row['Duration']} {row['Description']} {row['Spell Lists']} {row['Source']}", axis=1)

# Show example
print(finalSpells[['Name', 'searchable_text']].head())


               Name                                    searchable_text
181           Light  Light 0 Evocation  1 action  Touch  V, M (a fi...
240   Toll the Dead  Toll the Dead 0 Necromancy  1 action  60 feet ...
161     Thunderclap  Thunderclap 0 Evocation  1 action  Self (5-foo...
31          Message  Message 0 Transmutation  1 action  120 feet  V...
235  Create Bonfire  Create Bonfire 0 Conjuration  1 action  60 fee...


In [25]:
#Step IV.2 - Embed the searchable text

# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for all spells
embeddings = model.encode(finalSpells['searchable_text'].tolist(), show_progress_bar=True)

# Convert to numpy array for FAISS
embeddings_np = np.array(embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

In [26]:
#Step IV.3 - Create FAISS index

index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)

In [27]:
#Step IV.4 v2 - Create a function to query the model

def hybrid_search(query, level=None, spell_class=None, k=5):
    filtered_df = finalSpells.copy()

    # Optional filter for level
    if level is not None:
        print(f"Filtering for level {level}")
        filtered_df = filtered_df[filtered_df['Level'] == str(level)]
        print(f"Remaining after level filter: {len(filtered_df)}")

    # Optional filter for class (case-insensitive)
    if spell_class is not None:
        print(f"Filtering for class {spell_class}")
        mask = filtered_df['Spell Lists'].str.lower().str.contains(spell_class.lower(), na=False)
        filtered_df = filtered_df[mask]
        print(f"Remaining after class filter: {len(filtered_df)}")

    if filtered_df.empty:
        return pd.DataFrame()  # Return empty DataFrame for consistency

    # Step 2: Embed the filtered subset
    filtered_texts = filtered_df['searchable_text'].tolist()
    filtered_embeddings = model.encode(filtered_texts, convert_to_numpy=True)

    d = filtered_embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(filtered_embeddings)

    query_embedding = model.encode([query], convert_to_numpy=True)

    D, I = index.search(query_embedding, min(k, len(filtered_df)))

    results = filtered_df.iloc[I[0]].reset_index(drop=True)
    return results


In [28]:
results = hybrid_search("burn enemies", level=3, spell_class="sorcerer", k=5)

if not results.empty:
    print(results[['Name', 'Level','Description']])
else:
    print("No spells found for the given level and class.")


Filtering for level 3
Remaining after level filter: 75
Filtering for class sorcerer
Remaining after class filter: 34
                    Name Level  \
0               Fireball     3   
1           Flame Arrows     3   
2             Antagonize     3   
3    Ashardalon's Stride     3   
4  Melf's Minute Meteors     3   

                                         Description  
0  a bright streak flashes from your pointing fin...  
1  you touch a quiver containing arrows or bolts....  
2  you whisper magical words that antagonize one ...  
3  the billowing flames of a dragon blast from yo...  
4  you create six tiny meteors in your space. the...  


In [29]:
results.head()

,Name,Level,School,Casting Time,Range,Components,Duration,Description,Upcasting,Spell Lists,Source,URL,searchable_text
0,Fireball,3,evocation,1 action,150 feet,"V, S, M (a tiny ball of bat guano and sulfur)",Instantaneous,a bright streak flashes from your pointing fin...,when you cast this spell using a spell slot of...,"sorcerer, wizard",Player's Handbook,https://dnd5e.wikidot.com/spell:fireball,"Fireball 3 evocation 1 action 150 feet V, S..."
1,Flame Arrows,3,transmutation,1 action,Touch,"V, S","Concentration, up to 1 hour",you touch a quiver containing arrows or bolts....,when you cast this spell using a spell slot of...,"druid, ranger, sorcerer, wizard, artificer",Xanathar's Guide to Everything,https://dnd5e.wikidot.com/spell:flame-arrows,Flame Arrows 3 transmutation 1 action Touch ...
2,Antagonize,3,Enchantment,1 action,30 feet,"V, S, M (a playing card depicting a rogue)",Instantaneous,you whisper magical words that antagonize one ...,when you cast this spell using a spell slot of...,"bard, sorcerer, warlock, wizard",The Book of Many Things,https://dnd5e.wikidot.com/spell:antagonize,Antagonize 3 Enchantment 1 action 30 feet V...
3,Ashardalon's Stride,3,Transmutation,1 bonus action,Self,"V,S","Concentration, up to 1 minute",the billowing flames of a dragon blast from yo...,when you cast this spell using a spell slot of...,"artificer, ranger, sorcerer, wizard",Fizban's Treasury of Dragons,https://dnd5e.wikidot.com/spell:ashardalons-st...,Ashardalon's Stride 3 Transmutation 1 bonus a...
4,Melf's Minute Meteors,3,evocation,1 action,Self,"V, S, M (niter, sulfur, and pine tar formed i...","Concentration, up to 10 minutes",you create six tiny meteors in your space. the...,when you cast this spell using a spell slot of...,"sorcerer, wizard",Xanathar's Guide to Everything,https://dnd5e.wikidot.com/spell:melfs-minute-m...,Melf's Minute Meteors 3 evocation 1 action S...


In [30]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load model and tokenizer once
llm_model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
model_llm = AutoModelForSeq2SeqLM.from_pretrained(llm_model_name)

def llm_rewrite_query_hf(user_query):
    prompt = f"Rewrite this D&D spell search query to focus it for filtering a spell database:\n\n{user_query}\n\nRewritten query:"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=128, truncation=True)
    outputs = model_llm.generate(**inputs, max_length=50)
    rewritten = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return rewritten


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [31]:
def hybrid_search_with_llm_hf(query, level=None, spell_class=None, k=5):
    rewritten_query = llm_rewrite_query_hf(query)
    print(f"Original query: {query}")
    print(f"Rewritten query: {rewritten_query}")

    level = str(level) if level is not None else None
    spell_class = spell_class if spell_class != "" else None

    filtered_df = finalSpells.copy()
    if level:
        filtered_df = filtered_df[filtered_df['Level'] == level]
    if spell_class:
        filtered_df = filtered_df[filtered_df['Spell Lists'].str.contains(spell_class, case=False, na=False)]

    if filtered_df.empty:
        return f"No spells found for level={level} and class={spell_class}"

    filtered_texts = filtered_df['searchable_text'].tolist()
    filtered_embeddings = model.encode(filtered_texts, convert_to_numpy=True)

    d = filtered_embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(filtered_embeddings)

    query_embedding = model.encode([rewritten_query], convert_to_numpy=True)
    D, I = index.search(query_embedding, min(k, len(filtered_df)))

    results = filtered_df.iloc[I[0]].reset_index(drop=True)
    return results


In [32]:
def summarize_spells_with_llm(spells_df, user_query, max_spells=5):
    # Format the spells info into a plain text summary prompt
    spells_summary = ""
    for i, row in spells_df.head(max_spells).iterrows():
        spells_summary += f"- {row['Name']} (Level {row['Level']} {row['School']}): {row['Description'][:150].strip()}...\n"

    prompt = f"User query: {user_query}\n\nBased on the following D&D spells, provide a friendly summary highlighting the best options:\n\n{spells_summary}\nSummary:"

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model_llm.generate(**inputs, max_length=150, do_sample=True, top_p=0.95, temperature=0.7)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary


In [33]:
def hybrid_search_and_summarize(query, level=None, spell_class=None, k=5):
    results_df = hybrid_search_with_llm_hf(query, level, spell_class, k)

    if isinstance(results_df, str):
        # means "No spells found" message or error string
        return results_df

    summary = summarize_spells_with_llm(results_df, query, max_spells=k)
    return summary


In [34]:
summary = hybrid_search_and_summarize("burn enemies", level=2, spell_class="sorcerer", k=5)
print(summary)


Original query: burn enemies
Rewritten query: Burn enemies is a list of all the spells that are in the database.
Scorching Ray (Level 2 evocation): you create three rays of fire and hurl them at targets within range. you can hurl them at one target or several. make a ranged spell attack for each r...


In [35]:
import gradio as gr

def gradio_spell_search(query, level, spell_class, k):
    level = str(level) if level is not None else None
    spell_class = spell_class if spell_class != "" else None

    results = hybrid_search(query, level=level, spell_class=spell_class, k=int(k))

    if isinstance(results, str):  # No results found
        return results
    else:
        # Format results nicely
        return results[['Name', 'Level', 'School', 'Spell Lists', 'Description', 'Source', 'URL']].to_markdown(index=False)

# Gradio interface
gr.Interface(
    fn=gradio_spell_search,
    inputs=[
        gr.Textbox(label="Search Query", placeholder="Describe what you're looking for..."),
        gr.Number(label="Spell Level (optional)", value=None),
        gr.Textbox(label="Class Filter (optional)", placeholder="e.g. wizard, sorcerer..."),
        gr.Number(label="Top K Results", value=5)
    ],
    outputs=gr.Markdown(label="Matching Spells"),
    title="D&D 5e Spell Search",
    description="Search for spells based on a natural language query, with optional level and class filtering."
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b57b04bba7989fffb8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
